In [ ]:
# !pip install tifffile


In [6]:
import logging
import os
import sys
import tifffile
import pathlib
import tempfile
from glob import glob
from prettytable import PrettyTable
from collections import OrderedDict
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1" 

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

from PIL import Image
import cv2

import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
import torchvision
from torchvision import transforms
from torchvision.io import read_image
from torchvision.utils import make_grid, draw_segmentation_masks
from torchvision.transforms.functional import convert_image_dtype
import torchvision.transforms.functional as F


# from medpy.io import load as mha_load
import matplotlib.pyplot as plt

import monai
from monai.data import create_test_image_2d, list_data_collate, decollate_batch,pad_list_data_collate
from monai.inferers import sliding_window_inference, SimpleInferer
from monai.metrics import DiceMetric
from monai.networks.layers import Norm
from monai.utils import ThreadContainer


import warnings

from typing import Union
from monai.metrics.utils import do_metric_reduction, ignore_background
from monai.utils import MetricReduction

from monai.metrics import CumulativeIterationMetric

from monai.transforms import *
from monai.visualize import plot_2d_or_3d_image

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
# print(monai.config.print_config())

# log_file_name = f"experiments/experiment.log"
# logging.basicConfig(
#     level= logging.INFO,
#     format='%(message)s',
#     handlers=[
#         logging.FileHandler(log_file_name),
#         logging.StreamHandler()]
# )
# import utils
# import importlib as imp

# utils = imp.reload(utils)


from cmath import nan
import monai 
import logging
import os
import re
import sys
import time
import pickle
import tempfile
from glob import glob
from datetime import datetime
from  tqdm.notebook import tqdm
from prettytable import PrettyTable
from collections import OrderedDict
from skimage.filters import sato, meijering, frangi, hessian


import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
import torchvision
from torchvision import transforms
from torchvision.io import read_image
from torchvision.utils import make_grid, draw_segmentation_masks
from torchvision.transforms.functional import convert_image_dtype
import torchvision.transforms.functional as F

import cv2
import albumentations
from PIL import Image
# from medpy.io import load as mha_load
import matplotlib.pyplot as plt

import monai
from monai.data import create_test_image_2d, pad_list_data_collate, decollate_batch,pad_list_data_collate
from monai.inferers import sliding_window_inference, SimpleInferer
from monai.metrics import DiceMetric
from monai.optimizers import LearningRateFinder

import warnings

from typing import Union
from monai.metrics.utils import do_metric_reduction, ignore_background
from monai.utils import MetricReduction

from monai.metrics import CumulativeIterationMetric

from monai.transforms import *
from monai.visualize import plot_2d_or_3d_image

cuda:0


In [8]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('data/'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        break

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

data/train.csv
data/test_images/10078.tiff
data/train_annotations/11064.json
data/train_images/11448.tiff


In [9]:

def rle_decode(mask_rle, shape, color=1):
    '''
    mask_rle: run-length as string formated (start length)
    shape: (height,width) of array to return 
    Returns numpy array, 1 - mask, 0 - background
    '''
    s = mask_rle.split()
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
    starts -= 1
    ends = starts + lengths
    img = np.zeros(shape[0] * shape[1], dtype=np.float32)
    for lo, hi in zip(starts, ends):
        img[lo : hi] = color
    
    img = img.reshape(shape).T
#     print(img.shape)
    print("before", np.unique(img))
    img[img>0] =1 
    print("after", np.unique(img))
    return img

def read_image(image_path):
    return  tifffile.imread(image_path)
def load_segmentation_mask(mask_rle):
    return rle_decode(mask_rle, (3000,3000))

In [10]:
images =glob("data/train_images/*.tiff")
annotations = glob("data/train_annotations/*.json")
len(images)
len(annotations)

351

In [11]:
df = pd.read_csv("data/train.csv", dtype={"id":str})
df

,id,organ,data_source,img_height,img_width,pixel_size,tissue_thickness,rle,age,sex
0,10044,prostate,HPA,3000,3000,0.4,4,1459676 77 1462675 82 1465674 87 1468673 92 14...,37.0,Male
1,10274,prostate,HPA,3000,3000,0.4,4,715707 2 718705 8 721703 11 724701 18 727692 3...,76.0,Male
2,10392,spleen,HPA,3000,3000,0.4,4,1228631 20 1231629 24 1234624 40 1237623 47 12...,82.0,Male
3,10488,lung,HPA,3000,3000,0.4,4,3446519 15 3449517 17 3452514 20 3455510 24 34...,78.0,Male
4,10610,spleen,HPA,3000,3000,0.4,4,478925 68 481909 87 484893 105 487863 154 4908...,21.0,Female
...,...,...,...,...,...,...,...,...,...,...
346,9517,kidney,HPA,3000,3000,0.4,4,1611763 11 1614753 29 1617750 35 1620746 43 16...,61.0,Male
347,9769,kidney,HPA,3070,3070,0.4,4,4030400 28 4033466 34 4036526 48 4039594 54 40...,28.0,Male
348,9777,largeintestine,HPA,3000,3000,0.4,4,538473 13 541468 22 544463 30 547461 35 550459...,84.0,Male
349,9791,kidney,HPA,3000,3000,0.4,4,334733 33 337729 43 340729 43 343725 51 346723...,28.0,Male


## EDA 

### Data source, Age and Gender 

In [ ]:
plt.figure(figsize=(24,6))
plt.subplot(1,4,1)
plt.pie(df.groupby("data_source")["id"].count(), labels=df["data_source"].unique())
plt.title("Data source")
plt.subplot(1,4,2)
plt.pie(df["sex"].value_counts(), labels= df["sex"].unique())
plt.title("Gender")
plt.subplot(1,4,3)
plt.hist(x=df["age"],label="age")
plt.xlabel("age")
plt.ylabel("count")
plt.title("Age")
plt.subplot(1,4,4)
sns.countplot(x="organ", order=df["organ"].value_counts(ascending=True).index ,data=df)
plt.title("Organ")
plt.show()

### Image height width and pixel size

In [ ]:
plt.figure(figsize=(20,5))
plt.subplot(1,3,1)
plt.hist(x=df["img_height"])
plt.xlabel("img_height")
plt.ylabel("count")
plt.title("img_height")
plt.subplot(1,3,2)
plt.hist(x=df["img_width"])
plt.title("img_width")
plt.xlabel("img_width")
plt.ylabel("count")
plt.subplot(1,3,3)
plt.hist(x=df["pixel_size"])
plt.title("pixel_size")
plt.xlabel("pixel_size")
plt.ylabel("count")
plt.show()


### Sample Image

In [ ]:
plt.figure(figsize=(14,14))

sample_image = tifffile.imread(images[0])
plt.imshow(sample_image)
plt.show()

# Deep learning 

In [ ]:
image_path_prefix = "/kaggle/input/hubmap-organ-segmentation/train_images/"
image_path_sufix = ".tiff"
df["image_path"] = image_path_prefix+ df["id"] + image_path_sufix

train_df, val_df = train_test_split(df, test_size=0.2)

In [ ]:
train_files = [{"image": train_df.iloc[i,-1], "label":train_df.iloc[i,-4]} for i in range(len(train_df))]
val_files = [{"image": val_df.iloc[i,-1], "label":val_df.iloc[i,-4]} for i in range(len(val_df))]

print(len(train_files), len(val_files))

threshold =0
IMAGE_SIZE_1 = 512
IMAGE_SIZE_2 = 512   # 3072 = 1024 * 3
CLIP_LIMIT =(1,2)
TRAIN_BATCH_SIZE = 2
VAL_BATCH_SIZE = 1 # 16
TEST_BATCH_SIZE = 1

LEARNING_RATE = 1e-2

both_keys = ["image", "label"]


train_transforms = Compose([
    Lambdad(keys="image", func=read_image),
    Lambdad(keys="label", func=load_segmentation_mask),
    AddChanneld(keys="label"),
    AsChannelFirstd(keys="image"),
    EnsureTyped(keys=both_keys),
    Resized(both_keys, spatial_size=(IMAGE_SIZE_1, IMAGE_SIZE_2)),
    NormalizeIntensityd(keys=["image"], channel_wise=True), ## normalizing images only , subtrahend=mean, divisor=std 
])

val_transforms = Compose([
    Lambdad(keys="image", func=read_image),
    Lambdad(keys="label", func=load_segmentation_mask),
    AddChanneld(keys="label"),
    AsChannelFirstd(keys="image"),
    EnsureTyped(keys=both_keys),
    Resized(both_keys, spatial_size=(IMAGE_SIZE_1, IMAGE_SIZE_2)),
    NormalizeIntensityd(keys=["image"], channel_wise=True), ## normalizing images only , subtrahend=mean, divisor=std 

])




In [ ]:
train_ds = monai.data.Dataset(data=train_files, transform=train_transforms) #, num_workers=8
train_loader = DataLoader(
        train_ds,
        batch_size=8,
        shuffle=True,
        num_workers=1,
        collate_fn=pad_list_data_collate,
#         pin_memory=torch.cuda.is_available(),
    )

val_ds = monai.data.Dataset(data=val_files, transform=val_transforms) #, num_workers=8
val_loader = DataLoader(
        val_ds,
        batch_size=2,
        shuffle=False,
        num_workers=1,
        collate_fn=pad_list_data_collate,
#         pin_memory=torch.cuda.is_available(),
    )

In [ ]:
threshold = 0.5
for batch in train_loader:
    plt.figure(figsize=(16,16))
    image, label = batch["image"], batch["label"]
    image = image.permute(0,2,3,1)
    plt.subplot(1,2,1)
    plt.imshow(image[0])
    plt.title(image.shape)
    plt.subplot(1,2,2)
    label = label.permute(0,2,3,1)
#     label[label>threshold] =1  

    plt.imshow(label[0])
    plt.title(label.shape)
    plt.show()
    break
    
for batch in val_loader:
    plt.figure(figsize=(16,16))
    image, label = batch["image"], batch["label"]
    image = image.permute(0,2,3,1)
    plt.subplot(1,2,1)
    plt.imshow(image[0])
    plt.title(image.shape)
    plt.subplot(1,2,2)
    label = label.permute(0,2,3,1)
#     label[label>threshold] =1  
    plt.imshow(label[0])
    plt.title(str(label.shape) + str(np.unique(label)))
    plt.show()
    break

In [ ]:
dice_and_iou_metrics = []
thresholds = np.arange(start=0.5, stop=1, step=0.05)

## instantiating metric, 
dice_metric_fn = DiceMetric(include_background=True, reduction="mean", get_not_nans=False)
### UNET dont have activations in last layer, so using post transforms we pass model output and get sigmoid output
post_trans_fn = Compose([EnsureType(), Activations(sigmoid=True), AsDiscrete(threshold=0.5), KeepLargestConnectedComponent(applied_labels=[1], connectivity=1) ]) # 

def get_metrics_object_at_different_thresholds():
    ## instantiating metrics for all the thresholds
    for index , threshold in  enumerate(thresholds):
        threshold = round(threshold, ndigits=2)
        dice_metric = DiceMetric(include_background=True, reduction="mean", get_not_nans=False)
        post_trans = Compose([EnsureType(), Activations(sigmoid=True),AsDiscrete(threshold=threshold), KeepLargestConnectedComponent(applied_labels=[1], connectivity=1)]) #
        metric_transforms_dict = {"dice_metric":dice_metric, "post_trans":post_trans}
        dice_and_iou_metrics.append(metric_transforms_dict)
    return dice_and_iou_metrics
        
dice_and_iou_metrics = get_metrics_object_at_different_thresholds()
## method to calculate IoU and Dice metric for each thresholds
def calculate_mean_dice_and_iou_metric(y_pred, y):
    
    for index , threshold in  enumerate(thresholds):
        threshold = round(threshold, ndigits=2)
        post_trans = dice_and_iou_metrics[index]["post_trans"]
        val_outputs = [post_trans(i) for i in decollate_batch(y_pred)]
        dice_and_iou_metrics[index]["dice_metric"](y_pred=val_outputs, y=y)

## method to aggreate batch results and average them acorss all threshold   
def aggregate_dice_and_iou_metric():
    average_dice_metrics = torch.randn((len(thresholds)))
    t = PrettyTable(['treshold', 'dice metric'])

    for index , threshold in  enumerate(thresholds):
        threshold = round(threshold, ndigits=2)
        dice_metric = dice_and_iou_metrics[index]["dice_metric"].aggregate()
        average_dice_metrics[index] = dice_metric
        dice_and_iou_metrics[index]["dice_metric"].reset()
        
        t.add_row([threshold, round(dice_metric.item(), ndigits=4)])
    mean_average_dice_metric =  torch.mean(average_dice_metrics).item(),
    t.add_row(["Mean@0.5:0.95", round(mean_average_dice_metric[0], ndigits=4)])
    logging.info(t)
    return  mean_average_dice_metric[0]

    
def train_model(model, train_loader,val_loader, tensorboard_writer_path,device,learning_rate, epochs =1000):    
    loss_function = monai.losses.DiceCELoss(sigmoid=True, to_onehot_y=False,squared_pred=True, include_background=True)   
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    # optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9)
    # scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer,mode='max', factor=0.56 , patience=4, verbose=True)
    # scheduler = torch.optim.lr_scheduler.CyclicLR(optimizer, base_lr=6.6e-6, max_lr=6.6e-3, mode="triangular2",step_size_up=6*len(train_loader), cycle_momentum=False)#
    scheduler=torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=266, eta_min=0, last_epoch=-1)
    val_interval = 2
    best_metric = -1
    best_metric_epoch = -1
    epoch_loss_values = list()
    metric_values = list()
    writer = SummaryWriter(f"runs/{tensorboard_writer_path}")
    epochs = epochs
    pathlib.Path('experiments').mkdir(parents=True, exist_ok=True) 
    log_file_name = f"experiments/{tensorboard_writer_path}.log"
    logging.basicConfig(level= logging.INFO,format='%(message)s',handlers=[logging.FileHandler(log_file_name),logging.StreamHandler()])
    
  
    for epoch in tqdm(range(epochs)):
        logging.info("-" * 10)
        logging.info(f"epoch {epoch + 1}/{epochs}")
        print("-" * 10)
        print(f"epoch {epoch + 1}/{epochs}")
        model.train()
        epoch_loss = 0
        step = 0
        writer.add_scalar("learning_rate", optimizer.param_groups[0]['lr'], epoch + 1)
        # scheduler.step(best_metric)

        logging.info(f"learning_rate: { optimizer.param_groups[0]['lr']}")
        print(f"learning_rate: { optimizer.param_groups[0]['lr']}")
        for batch_data in train_loader:
            step += 1
            inputs, labels = batch_data["image"].to(device), batch_data["label"].to(device)
            optimizer.zero_grad()
            
            # outputs = sliding_window_inference(inputs, (1024, 1024), 3, model)
            outputs = model(inputs)
            loss = loss_function(outputs, labels)
            if loss.item() == np.nan:
                print(outputs)
            
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()
            epoch_len = (len(train_loader)*train_loader.batch_size )// train_loader.batch_size
            
            scheduler.step()  # for cyclic learning rate
            
            if step % 2== 0:
                logging.info(f"{step}/{epoch_len}, train_loss: {loss.item():.4f}")
                print(f"{step}/{epoch_len}, train_loss: {loss.item():.4f}")
            
        epoch_loss /= step
        epoch_loss_values.append(epoch_loss)
        logging.info(f"epoch {epoch + 1} average loss: {epoch_loss:.4f}")
        
        if  (epoch + 1) % val_interval == 0:
            print("validation")
            model.eval()
            with torch.no_grad():
                val_images = None
                val_labels = None
                val_outputs = None
                val_epoch_loss = 0
                val_steps = 0
                for val_data in val_loader:
                    
                    val_steps +=1
                    val_images, val_labels = val_data["image"].to(device), val_data["label"].to(device)
                    # with torch.cuda.amp.autocast(enabled=False):
                    # val_outputs = sliding_window_inference(val_images, (1024, 1024), 3, model)
                    val_outputs =  model(val_images)
                    val_loss = loss_function(val_outputs, val_labels)
                    val_epoch_loss+=val_loss
                    raw_val_outputs = val_outputs.detach()
                    val_outputs = [post_trans_fn(i) for i in decollate_batch(val_outputs)]
                    
                    calculate_mean_dice_and_iou_metric(y_pred=raw_val_outputs, y=val_labels )
                    
                # plot the last model output as GIF image in TensorBoard with the corresponding image and label
                    if val_steps == 1:
                        for i in range(VAL_BATCH_SIZE):
                            if epoch ==1:
                                plot_2d_or_3d_image(val_images, epoch + 1, writer, index=i, tag="val"+str(i)+"/image")
                                plot_2d_or_3d_image(val_labels, epoch + 1, writer, index=i, tag="val"+str(i)+"/label")
                            plot_2d_or_3d_image(val_outputs, epoch + 1, writer, index=i, tag="val"+str(i)+"/output")
                # aggregate the final mean dice result
                
                metric = aggregate_dice_and_iou_metric()
                val_epoch_loss /= val_steps
                metric_values.append(metric)
                if metric > best_metric:
                    best_metric = metric
                    best_metric_epoch = epoch + 1
                    pathlib.Path('models').mkdir(parents=True, exist_ok=True) 
                    torch.save(model, f"models/{tensorboard_writer_path}.pth")
                    logging.info("saved new best metric model")
                    print("saved new best metric model")
                logging.info(
                    "current epoch: {} current mean DICE @ 0.95: {:.4f} best mean DICE @ 0.95: {:.4f} at epoch {}".format(
                        epoch + 1, metric, best_metric, best_metric_epoch
                    )
                )
                
                print(
                    "current epoch: {} current mean DICE @ 0.95: {:.4f} best mean DICE @ 0.95: {:.4f} at epoch {}".format(
                        epoch + 1, metric, best_metric, best_metric_epoch
                    )
                )
                writer.add_scalar("val_mean_DICE@0.5:0.95", metric, epoch + 1)
                writer.add_scalars("loss", {"train":epoch_loss, "validation": val_epoch_loss}, epoch + 1)
                


        
        
        
    logging.info(f"train completed, best_metric: {best_metric:.4f} at epoch: {best_metric_epoch}")
    writer.close()
    

In [ ]:
largest_connector_model  = monai.networks.nets.UNet(
    spatial_dims=2,
    in_channels=3,
    out_channels=1,
    channels=(8, 16, 32, 64, 128),
    strides=(2, 2, 2, 2),
    dropout=0.36,
    num_res_units=1,
    norm="batch",
).to(device)
experiment_name = "testing"


In [ ]:
train_model(
    model=largest_connector_model,
    train_loader=train_loader,
    val_loader=val_loader,
    tensorboard_writer_path=experiment_name,
    device=device,
    learning_rate=2.6e-2
    )

In [ ]:
logging.info("hello")

In [ ]:
!tensorboard --logdir